In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib as mplt
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from IPython.core.interactiveshell import InteractiveShell
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import explained_variance_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
import matplotlib.dates as mdates
%matplotlib inline
import seaborn as sns
import math
import gc
import ipaddress
from urllib.parse import urlparse
from tldextract import extract
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

from data_science_utils import dataframe as df_utils
from data_science_utils import models as model_utils
from data_science_utils.dataframe import column as column_utils


from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix,classification_report

import lightgbm as lgb

np.set_printoptions(threshold=np.nan)



plt.rcParams["figure.figsize"] = (24,4)

from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score
import missingno as msno
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import datasets
from sklearn.decomposition import PCA
import datetime
from scipy import signal
import matplotlib.pyplot as plt
from datetime import timedelta
from sklearn import linear_model



InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_seq_items', None)
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import warnings
warnings.filterwarnings('ignore')
import dask.dataframe as dd

In [2]:
df_train = pd.read_hdf("data/train_1.h5","table")
gc.collect()

56

In [3]:
keys = [0,20,25,30,45,50,55,70,75,80,100]
values = np.percentile(df_train.index,keys).astype(int)
pcs = dict(zip(keys, values))
pcs

{0: 0,
 20: 36980777,
 25: 46225972,
 30: 55471166,
 45: 83206750,
 50: 92451944,
 55: 101697138,
 70: 129432722,
 75: 138677916,
 80: 147923111,
 100: 184903889}

In [4]:
gc.collect()
# p1 = df_train[df_train.index<pcs[30]]
# p1.index.max()
# p2 = df_train[(df_train.index>pcs[20])&(df_train.index<pcs[55])]
# p2.index.max()
# p3 = df_train[(df_train.index>pcs[45])&(df_train.index<pcs[80])]
# p3.index.max()
p4 = df_train[df_train.index>pcs[70]]
p4.index.max()

df_train = None
gc.collect()

# p1 = dd.from_pandas(p1,npartitions=64)
# p2 = dd.from_pandas(p2,npartitions=64)
# p3 = dd.from_pandas(p3,npartitions=64)
p4 = dd.from_pandas(p4,npartitions=64)
gc.collect()

0

184903889

0

18

In [5]:


ALL_NEXT_CLICKS = [
    
    {'groupby': ['ip']},
    {'groupby': ['app']},
    {'groupby': ['device']},
    {'groupby': ['ip', 'app']},
    {'groupby': ['ip', 'channel']},
    {'groupby': ['ip', 'os']},
    # V3
    {'groupby': ['ip', 'app', 'device', 'os', 'channel']},
    {'groupby': ['ip', 'os', 'device']},
    {'groupby': ['ip', 'os', 'device', 'app']},
    
    #{'groupby': ['ip','hour']},
    #{'groupby': ['ip', 'app', 'device', 'os', 'channel','hour']},
    {'groupby': ['ip', 'os', 'device','hour']},
    {'groupby': ['ip', 'os', 'device', 'app','hour']},
    {'groupby': ['ip', 'app','hour']},
    {'groupby': ['ip', 'channel','hour']},
    {'groupby': ['ip', 'os','hour']},
    {'groupby': ['app', 'device', 'os', 'channel']},
    {'groupby': ['app', 'device', 'os']},
    {'groupby': ['app', 'device', 'os','hour']}
]

In [6]:


def add_time_to_next_click(GROUP_BY_NEXT_CLICKS,df):
    gc.collect()
    for spec in GROUP_BY_NEXT_CLICKS:
        next_click = 'nc_{}'.format('_'.join(spec['groupby']))
        grad_click = 'gc_{}'.format('_'.join(spec['groupby']))
        all_features = spec['groupby'] + ['click_time']

        print(f"Grouping by {spec['groupby']}, and saving time to next click in: {next_click}")
        # group = df[all_features].groupby(spec['groupby']).click_time  
        # gp = group.transform(lambda x: x.diff().shift(-1))
        def differ(x):
            return x.diff().shift(-1)
        
        def grad(x):
            if(x.size<=1):
                return pd.Series(np.nan,index=x.index)
            return pd.Series(np.gradient(x,axis=0),index=x.index)
        group = df.groupby(spec['groupby']).click_time
        gp1 = group.apply(differ).compute()
        gp2 = group.apply(grad).compute()
        
        df[next_click] = gp1
        df[grad_click] = gp2
        
        gp=None
        del gp
        group = None
        del group
        gc.collect()
    


In [7]:
add_time_to_next_click(ALL_NEXT_CLICKS,p4)
gc.collect()

Grouping by ['ip'], and saving time to next click in: nc_ip
Grouping by ['app'], and saving time to next click in: nc_app
Grouping by ['device'], and saving time to next click in: nc_device
Grouping by ['ip', 'app'], and saving time to next click in: nc_ip_app
Grouping by ['ip', 'channel'], and saving time to next click in: nc_ip_channel
Grouping by ['ip', 'os'], and saving time to next click in: nc_ip_os
Grouping by ['ip', 'app', 'device', 'os', 'channel'], and saving time to next click in: nc_ip_app_device_os_channel
Grouping by ['ip', 'os', 'device'], and saving time to next click in: nc_ip_os_device
Grouping by ['ip', 'os', 'device', 'app'], and saving time to next click in: nc_ip_os_device_app
Grouping by ['ip', 'os', 'device', 'hour'], and saving time to next click in: nc_ip_os_device_hour
Grouping by ['ip', 'os', 'device', 'app', 'hour'], and saving time to next click in: nc_ip_os_device_app_hour
Grouping by ['ip', 'app', 'hour'], and saving time to next click in: nc_ip_app_hour

14

In [8]:
gc.collect()
p4.to_hdf('intermediates/train_2_part4.h5',"table",mode='w',format='table')
gc.collect()
print("train_2_part4.h5 done")
df_utils.count_nulls(p4)

0

['intermediates/train_2_part4.h5',
 'intermediates/train_2_part4.h5',
 'intermediates/train_2_part4.h5',
 'intermediates/train_2_part4.h5',
 'intermediates/train_2_part4.h5',
 'intermediates/train_2_part4.h5',
 'intermediates/train_2_part4.h5',
 'intermediates/train_2_part4.h5',
 'intermediates/train_2_part4.h5',
 'intermediates/train_2_part4.h5',
 'intermediates/train_2_part4.h5',
 'intermediates/train_2_part4.h5',
 'intermediates/train_2_part4.h5',
 'intermediates/train_2_part4.h5',
 'intermediates/train_2_part4.h5',
 'intermediates/train_2_part4.h5',
 'intermediates/train_2_part4.h5',
 'intermediates/train_2_part4.h5',
 'intermediates/train_2_part4.h5',
 'intermediates/train_2_part4.h5',
 'intermediates/train_2_part4.h5',
 'intermediates/train_2_part4.h5',
 'intermediates/train_2_part4.h5',
 'intermediates/train_2_part4.h5',
 'intermediates/train_2_part4.h5',
 'intermediates/train_2_part4.h5',
 'intermediates/train_2_part4.h5',
 'intermediates/train_2_part4.h5',
 'intermediates/trai

10185

train_2_part4.h5 done


ValueError: DataFrame constructor not properly called!